In [3]:
import argparse
import os
import time
import shutil

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
     

import torchvision
import torchvision.transforms as transforms

from models import *


global best_prec
use_gpu = torch.cuda.is_available()
print('=> Building model...')
    
    
    
batch_size = 128
#model_name = "VGG16_quant"
#model = VGG16_quant()
model_name = "resnet20_quant"
model = resnet20_quant()
print(model)

normalize = transforms.Normalize(mean=[0.491, 0.482, 0.447], std=[0.247, 0.243, 0.262])


train_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ]))
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)


test_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        normalize,
    ]))

testloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)


print_freq = 100 # every 100 batches, accuracy printed. Here, each batch includes "batch_size" data points
# CIFAR10 has 50,000 training data, and 10,000 validation data.

def train(trainloader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    model.train()

    end = time.time()
    for i, (input, target) in enumerate(trainloader):
        # measure data loading time
        data_time.update(time.time() - end)

        input, target = input.cuda(), target.cuda()

        # compute output
        output = model(input)
        loss = criterion(output, target)

        # measure accuracy and record loss
        prec = accuracy(output, target)[0]
        losses.update(loss.item(), input.size(0))
        top1.update(prec.item(), input.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()


        if i % print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   epoch, i, len(trainloader), batch_time=batch_time,
                   data_time=data_time, loss=losses, top1=top1))

            

def validate(val_loader, model, criterion ):
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    with torch.no_grad():
        for i, (input, target) in enumerate(val_loader):
         
            input, target = input.cuda(), target.cuda()

            # compute output
            output = model(input)
            loss = criterion(output, target)

            # measure accuracy and record loss
            prec = accuracy(output, target)[0]
            losses.update(loss.item(), input.size(0))
            top1.update(prec.item(), input.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % print_freq == 0:  # This line shows how frequently print out the status. e.g., i%5 => every 5 batch, prints out
                print('Test: [{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   i, len(val_loader), batch_time=batch_time, loss=losses,
                   top1=top1))

    print(' * Prec {top1.avg:.3f}% '.format(top1=top1))
    return top1.avg


def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res


class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

        
def save_checkpoint(state, is_best, fdir):
    filepath = os.path.join(fdir, 'checkpoint.pth')
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(fdir, 'model_best.pth.tar'))


def adjust_learning_rate(optimizer, epoch):
    """For resnet, the lr starts from 0.1, and is divided by 10 at 80 and 120 epochs"""
    #adjust_list = [150, 225]
    adjust_list = [80, 120]
    if epoch in adjust_list:
        for param_group in optimizer.param_groups:
            param_group['lr'] = param_group['lr'] * 0.1        

#model = nn.DataParallel(model).cuda()
#all_params = checkpoint['state_dict']
#model.load_state_dict(all_params, strict=False)
#criterion = nn.CrossEntropyLoss().cuda()
#validate(testloader, model, criterion)

=> Building model...
ResNet_Cifar(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): QuantConv2d(
        16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
        (weight_quant): weight_quantize_fn()
      )
      (conv2): QuantConv2d(
        16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
        (weight_quant): weight_quantize_fn()
      )
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): QuantConv2d(
        16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
        (weight_quant): weight_quantize_fn()

In [4]:
# This cell won't be given, but students will complete the training

lr = 4e-2
weight_decay = 1e-4
epochs = 100
best_prec = 0

#model = nn.DataParallel(model).cuda()
model.cuda()
criterion = nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=weight_decay)
#cudnn.benchmark = True

if not os.path.exists('result'):
    os.makedirs('result')
fdir = 'result/'+str(model_name)
if not os.path.exists(fdir):
    os.makedirs(fdir)
        

for epoch in range(0, epochs):
    adjust_learning_rate(optimizer, epoch)

    train(trainloader, model, criterion, optimizer, epoch)
    
    # evaluate on test set
    print("Validation starts")
    prec = validate(testloader, model, criterion)

    # remember best precision and save checkpoint
    is_best = prec > best_prec
    best_prec = max(prec,best_prec)
    print('best acc: {:1f}'.format(best_prec))
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict': model.state_dict(),
        'best_prec': best_prec,
        'optimizer': optimizer.state_dict(),
    }, is_best, fdir)

Epoch: [0][0/391]	Time 0.635 (0.635)	Data 0.517 (0.517)	Loss 2.4757 (2.4757)	Prec 8.594% (8.594%)
Epoch: [0][100/391]	Time 0.053 (0.052)	Data 0.002 (0.007)	Loss 1.6464 (1.9506)	Prec 35.156% (25.387%)
Epoch: [0][200/391]	Time 0.044 (0.049)	Data 0.002 (0.004)	Loss 1.5246 (1.7952)	Prec 43.750% (32.027%)
Epoch: [0][300/391]	Time 0.043 (0.048)	Data 0.002 (0.004)	Loss 1.4017 (1.6901)	Prec 54.688% (36.566%)
Validation starts
Test: [0/79]	Time 0.322 (0.322)	Loss 1.5288 (1.5288)	Prec 47.656% (47.656%)
 * Prec 46.670% 
best acc: 46.670000
Epoch: [1][0/391]	Time 0.587 (0.587)	Data 0.530 (0.530)	Loss 1.2948 (1.2948)	Prec 51.562% (51.562%)
Epoch: [1][100/391]	Time 0.054 (0.052)	Data 0.003 (0.007)	Loss 1.2083 (1.2483)	Prec 55.469% (54.525%)
Epoch: [1][200/391]	Time 0.048 (0.049)	Data 0.002 (0.005)	Loss 1.2684 (1.1988)	Prec 53.125% (56.685%)
Epoch: [1][300/391]	Time 0.044 (0.047)	Data 0.002 (0.004)	Loss 1.3209 (1.1592)	Prec 52.344% (58.101%)
Validation starts
Test: [0/79]	Time 0.298 (0.298)	Loss 1.07

Epoch: [15][200/391]	Time 0.048 (0.049)	Data 0.002 (0.004)	Loss 0.3781 (0.4200)	Prec 88.281% (85.343%)
Epoch: [15][300/391]	Time 0.047 (0.048)	Data 0.002 (0.003)	Loss 0.3992 (0.4174)	Prec 85.938% (85.450%)
Validation starts
Test: [0/79]	Time 0.340 (0.340)	Loss 0.4975 (0.4975)	Prec 84.375% (84.375%)
 * Prec 81.780% 
best acc: 82.140000
Epoch: [16][0/391]	Time 0.521 (0.521)	Data 0.456 (0.456)	Loss 0.4428 (0.4428)	Prec 85.938% (85.938%)
Epoch: [16][100/391]	Time 0.038 (0.051)	Data 0.002 (0.007)	Loss 0.2799 (0.3914)	Prec 90.625% (86.293%)
Epoch: [16][200/391]	Time 0.050 (0.049)	Data 0.003 (0.004)	Loss 0.3034 (0.3992)	Prec 88.281% (86.031%)
Epoch: [16][300/391]	Time 0.051 (0.048)	Data 0.003 (0.004)	Loss 0.3618 (0.4026)	Prec 86.719% (85.849%)
Validation starts
Test: [0/79]	Time 0.258 (0.258)	Loss 0.5720 (0.5720)	Prec 78.906% (78.906%)
 * Prec 81.990% 
best acc: 82.140000
Epoch: [17][0/391]	Time 0.513 (0.513)	Data 0.454 (0.454)	Loss 0.3349 (0.3349)	Prec 85.156% (85.156%)
Epoch: [17][100/391]	

Epoch: [30][300/391]	Time 0.033 (0.046)	Data 0.002 (0.004)	Loss 0.2007 (0.2923)	Prec 95.312% (89.883%)
Validation starts
Test: [0/79]	Time 0.456 (0.456)	Loss 0.4597 (0.4597)	Prec 85.156% (85.156%)
 * Prec 85.180% 
best acc: 86.170000
Epoch: [31][0/391]	Time 0.564 (0.564)	Data 0.498 (0.498)	Loss 0.2876 (0.2876)	Prec 89.844% (89.844%)
Epoch: [31][100/391]	Time 0.045 (0.049)	Data 0.002 (0.007)	Loss 0.2554 (0.2826)	Prec 92.969% (90.509%)
Epoch: [31][200/391]	Time 0.045 (0.047)	Data 0.002 (0.004)	Loss 0.2905 (0.2918)	Prec 89.844% (89.887%)
Epoch: [31][300/391]	Time 0.048 (0.047)	Data 0.002 (0.004)	Loss 0.2881 (0.2983)	Prec 88.281% (89.636%)
Validation starts
Test: [0/79]	Time 0.320 (0.320)	Loss 0.4137 (0.4137)	Prec 85.156% (85.156%)
 * Prec 85.900% 
best acc: 86.170000
Epoch: [32][0/391]	Time 0.938 (0.938)	Data 0.873 (0.873)	Loss 0.2668 (0.2668)	Prec 88.281% (88.281%)
Epoch: [32][100/391]	Time 0.048 (0.055)	Data 0.002 (0.011)	Loss 0.2651 (0.2885)	Prec 89.062% (89.782%)
Epoch: [32][200/391]	

Validation starts
Test: [0/79]	Time 0.318 (0.318)	Loss 0.3463 (0.3463)	Prec 88.281% (88.281%)
 * Prec 85.250% 
best acc: 86.800000
Epoch: [46][0/391]	Time 0.736 (0.736)	Data 0.667 (0.667)	Loss 0.1734 (0.1734)	Prec 95.312% (95.312%)
Epoch: [46][100/391]	Time 0.047 (0.053)	Data 0.002 (0.009)	Loss 0.2154 (0.2333)	Prec 91.406% (91.569%)
Epoch: [46][200/391]	Time 0.039 (0.049)	Data 0.002 (0.005)	Loss 0.1002 (0.2426)	Prec 97.656% (91.340%)
Epoch: [46][300/391]	Time 0.039 (0.048)	Data 0.002 (0.004)	Loss 0.2656 (0.2389)	Prec 91.406% (91.541%)
Validation starts
Test: [0/79]	Time 0.414 (0.414)	Loss 0.3031 (0.3031)	Prec 88.281% (88.281%)
 * Prec 86.130% 
best acc: 86.800000
Epoch: [47][0/391]	Time 0.781 (0.781)	Data 0.715 (0.715)	Loss 0.1189 (0.1189)	Prec 97.656% (97.656%)
Epoch: [47][100/391]	Time 0.083 (0.089)	Data 0.002 (0.009)	Loss 0.2719 (0.2220)	Prec 87.500% (91.979%)
Epoch: [47][200/391]	Time 0.079 (0.086)	Data 0.003 (0.006)	Loss 0.2081 (0.2300)	Prec 92.188% (91.912%)
Epoch: [47][300/391]	

 * Prec 87.130% 
best acc: 87.530000
Epoch: [61][0/391]	Time 0.530 (0.530)	Data 0.464 (0.464)	Loss 0.2754 (0.2754)	Prec 90.625% (90.625%)
Epoch: [61][100/391]	Time 0.047 (0.048)	Data 0.002 (0.006)	Loss 0.2271 (0.2028)	Prec 90.625% (92.713%)
Epoch: [61][200/391]	Time 0.048 (0.046)	Data 0.002 (0.004)	Loss 0.1857 (0.2072)	Prec 91.406% (92.576%)
Epoch: [61][300/391]	Time 0.045 (0.045)	Data 0.002 (0.003)	Loss 0.1326 (0.2095)	Prec 96.094% (92.535%)
Validation starts
Test: [0/79]	Time 0.412 (0.412)	Loss 0.2258 (0.2258)	Prec 89.062% (89.062%)
 * Prec 87.320% 
best acc: 87.530000
Epoch: [62][0/391]	Time 0.637 (0.637)	Data 0.570 (0.570)	Loss 0.1830 (0.1830)	Prec 93.750% (93.750%)
Epoch: [62][100/391]	Time 0.050 (0.051)	Data 0.002 (0.008)	Loss 0.2203 (0.1945)	Prec 90.625% (93.162%)
Epoch: [62][200/391]	Time 0.043 (0.049)	Data 0.002 (0.005)	Loss 0.2821 (0.1992)	Prec 89.844% (92.977%)
Epoch: [62][300/391]	Time 0.041 (0.047)	Data 0.002 (0.004)	Loss 0.2063 (0.2044)	Prec 92.188% (92.753%)
Validation s

Epoch: [76][100/391]	Time 0.079 (0.088)	Data 0.002 (0.008)	Loss 0.2000 (0.1951)	Prec 93.750% (92.976%)
Epoch: [76][200/391]	Time 0.079 (0.086)	Data 0.003 (0.006)	Loss 0.1972 (0.1904)	Prec 91.406% (93.202%)
Epoch: [76][300/391]	Time 0.083 (0.084)	Data 0.002 (0.005)	Loss 0.3078 (0.1920)	Prec 89.062% (93.122%)
Validation starts
Test: [0/79]	Time 0.555 (0.555)	Loss 0.4028 (0.4028)	Prec 86.719% (86.719%)
 * Prec 87.680% 
best acc: 87.900000
Epoch: [77][0/391]	Time 0.955 (0.955)	Data 0.888 (0.888)	Loss 0.1882 (0.1882)	Prec 90.625% (90.625%)
Epoch: [77][100/391]	Time 0.084 (0.090)	Data 0.002 (0.011)	Loss 0.2464 (0.1815)	Prec 92.969% (93.510%)
Epoch: [77][200/391]	Time 0.077 (0.086)	Data 0.002 (0.007)	Loss 0.1531 (0.1847)	Prec 94.531% (93.385%)
Epoch: [77][300/391]	Time 0.080 (0.085)	Data 0.002 (0.005)	Loss 0.1112 (0.1908)	Prec 96.094% (93.233%)
Validation starts
Test: [0/79]	Time 0.601 (0.601)	Loss 0.2829 (0.2829)	Prec 92.188% (92.188%)
 * Prec 86.730% 
best acc: 87.900000
Epoch: [78][0/391]	

Epoch: [91][200/391]	Time 0.040 (0.047)	Data 0.002 (0.005)	Loss 0.0755 (0.0737)	Prec 97.656% (97.520%)
Epoch: [91][300/391]	Time 0.047 (0.047)	Data 0.002 (0.004)	Loss 0.0996 (0.0741)	Prec 96.094% (97.511%)
Validation starts
Test: [0/79]	Time 0.418 (0.418)	Loss 0.3102 (0.3102)	Prec 92.188% (92.188%)
 * Prec 90.230% 
best acc: 90.530000
Epoch: [92][0/391]	Time 0.447 (0.447)	Data 0.362 (0.362)	Loss 0.1882 (0.1882)	Prec 94.531% (94.531%)
Epoch: [92][100/391]	Time 0.041 (0.048)	Data 0.002 (0.005)	Loss 0.0613 (0.0741)	Prec 98.438% (97.703%)
Epoch: [92][200/391]	Time 0.042 (0.046)	Data 0.002 (0.004)	Loss 0.0431 (0.0720)	Prec 99.219% (97.625%)
Epoch: [92][300/391]	Time 0.043 (0.046)	Data 0.002 (0.003)	Loss 0.0756 (0.0717)	Prec 96.094% (97.620%)
Validation starts
Test: [0/79]	Time 0.326 (0.326)	Loss 0.3048 (0.3048)	Prec 91.406% (91.406%)
 * Prec 90.260% 
best acc: 90.530000
Epoch: [93][0/391]	Time 0.529 (0.529)	Data 0.462 (0.462)	Loss 0.0465 (0.0465)	Prec 97.656% (97.656%)
Epoch: [93][100/391]	

In [ ]:
# HW

#  1. Train with 4 bits for both weight and activation to achieve >90% accuracy
#  2. Find x_int and w_int for the 2nd convolution layer
#  3. Check the recovered psum has similar value to the un-quantized original psum
#     (such as example 1 in W3S2)

In [6]:
#PATH = "result/VGG16_quant/model_best.pth.tar"
PATH = "result/resnet20_quant/model_best.pth.tar"
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['state_dict'])
device = torch.device("cuda") 

model.cuda()
model.eval()

test_loss = 0
correct = 0

with torch.no_grad():
    for data, target in testloader:
        data, target = data.to(device), target.to(device) # loading to GPU
        output = model(data)
        pred = output.argmax(dim=1, keepdim=True)  
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(testloader.dataset)

print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct, len(testloader.dataset),
        100. * correct / len(testloader.dataset)))


Test set: Accuracy: 9053/10000 (91%)



In [7]:
#send an input and grap the value by using prehook like HW3
class SaveOutput:
    def __init__(self):
        self.outputs = []
    def __call__(self, module, module_in):
        self.outputs.append(module_in)
    def clear(self):
        self.outputs = []  
        
######### Save inputs from selected layer ##########
save_output = SaveOutput()

for layer in model.modules():
    if isinstance(layer, torch.nn.Conv2d):
        print("prehooked")
        layer.register_forward_pre_hook(save_output)       ## Input for the module will be grapped       
####################################################

dataiter = iter(trainloader)
images, labels = dataiter.next()
images = images.to(device)
out = model(images)
print("1st convolution's input size:", save_output.outputs[0][0].size())
print("2nd convolution's input size:", save_output.outputs[1][0].size())

prehooked
prehooked
prehooked
prehooked
prehooked
prehooked
prehooked
prehooked
prehooked
prehooked
prehooked
prehooked
prehooked
prehooked
prehooked
prehooked
prehooked
prehooked
prehooked
prehooked
prehooked
1st convolution's input size: torch.Size([128, 3, 32, 32])
2nd convolution's input size: torch.Size([128, 16, 32, 32])


In [8]:
conv2= model.layer1[0].conv1
#print(conv2.weight_q)
#print(conv2.weight_quant.wgt_alpha.data.item())
#print(conv2.weight_quant.wgt_alpha.data.item()/ (2**4 - 1))

In [9]:
w_bit = 4
#weight_q = model.features[3].weight_q # quantized value is stored during the training

weight_q = conv2.weight_q
w_alpha = conv2.weight_quant.wgt_alpha.data.item()
w_delta = w_alpha / (2**(w_bit - 1) - 1)
weight_int = weight_q / w_delta
print(weight_int) # you should see clean integer numbers

tensor([[[[ 0.0000,  1.0000, -2.0000],
          [ 3.0000,  0.0000, -4.0000],
          [-4.0000, -7.0000, -3.0000]],

         [[ 3.0000,  1.0000,  1.0000],
          [-0.0000, -5.0000, -4.0000],
          [-2.0000, -1.0000, -1.0000]],

         [[ 2.0000, -2.0000, -4.0000],
          [ 2.0000, -5.0000,  6.0000],
          [-2.0000,  2.0000, -3.0000]],

         ...,

         [[-7.0000, -2.0000,  1.0000],
          [ 1.0000,  0.0000,  3.0000],
          [ 5.0000,  4.0000, -1.0000]],

         [[ 5.0000,  6.0000, -4.0000],
          [ 3.0000, -1.0000, -4.0000],
          [-4.0000, -7.0000, -6.0000]],

         [[-5.0000,  0.0000,  1.0000],
          [-0.0000,  6.0000, -1.0000],
          [-3.0000,  0.0000,  0.0000]]],


        [[[-3.0000,  4.0000,  7.0000],
          [-4.0000,  2.0000,  5.0000],
          [-2.0000,  3.0000,  1.0000]],

         [[-5.0000, -2.0000,  3.0000],
          [-3.0000, -1.0000, -0.0000],
          [ 2.0000,  1.0000,  1.0000]],

         [[-0.0000, -0.0000,  2

In [ ]:
#print(save_output.outputs[1][0])
#print(conv2.act_alpha.data.item())


In [10]:
x_bit = 4
x =  save_output.outputs[1][0] # input of the 2nd conv layer
x_alpha  = conv2.act_alpha.data.item()
x_delta = x_alpha / (2**x_bit - 1)

act_quant_fn = act_quantization(x_bit) # define the quantization function
x_q = act_quant_fn(x, x_alpha)         # create the quantized value for x

x_int = x_q / x_delta
print(x_int) # you should see clean integer numbers 

tensor([[[[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  2.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  3.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  3.0000],
          ...,
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  3.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  3.0000],
          [ 1.0000,  2.0000,  2.0000,  ...,  0.0000,  3.0000,  5.0000]],

         [[ 7.0000,  3.0000,  3.0000,  ..., 10.0000,  6.0000,  0.0000],
          [ 6.0000,  2.0000,  2.0000,  ..., 12.0000,  6.0000,  0.0000],
          [ 7.0000,  2.0000,  2.0000,  ..., 15.0000,  7.0000,  0.0000],
          ...,
          [ 7.0000,  2.0000,  2.0000,  ..., 15.0000, 11.0000,  0.0000],
          [ 7.0000,  2.0000,  2.0000,  ..., 15.0000, 11.0000,  0.0000],
          [ 2.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

         [[ 0.0000,  5.0000,  5.0000,  ..., 15.0000,  0.0000,  0.0000],
          [ 0.0000,  5.0000,  

In [11]:
conv_int = torch.nn.Conv2d(in_channels = 16, out_channels=16, kernel_size = 3, bias = False)
conv_int.weight = torch.nn.parameter.Parameter(weight_int)

output_int =  conv_int(x_int)
output_recovered = output_int * x_delta * w_delta
print(output_recovered)

tensor([[[[-3.1312e-01, -3.1312e-01,  6.2624e-01,  ..., -1.0646e+01,
           -1.9961e+01, -5.9492e+00],
          [-3.9140e+00, -1.5656e+00, -2.3484e-01,  ..., -1.0646e+01,
           -3.3425e+01, -1.2525e+01],
          [-2.1918e+00, -7.8279e-01, -1.6439e+00,  ..., -1.2603e+01,
           -3.2408e+01, -1.4169e+01],
          ...,
          [-4.8533e+00, -3.9140e+00, -1.1742e+00,  ..., -5.0882e+00,
           -3.7966e+01, -1.4482e+01],
          [-1.1742e+00, -1.1742e+00, -9.3935e-01,  ..., -6.9669e+00,
           -3.3190e+01, -1.7613e+01],
          [-1.0803e+01, -1.4951e+01, -1.3307e+01,  ..., -3.5304e+01,
           -5.9101e+01, -2.1605e+01]],

         [[-1.1272e+01, -7.0451e+00, -8.5325e+00,  ..., -3.3660e+00,
           -4.0705e+00, -1.6752e+01],
          [-6.5755e+00,  3.9140e-01, -1.5656e+00,  ..., -7.1234e+00,
           -1.1820e+01, -1.6987e+01],
          [-7.9845e+00,  3.9140e-01,  1.2525e+00,  ..., -6.5755e+00,
           -1.3542e+01, -2.0196e+01],
          ...,
     

In [ ]:
#### input floating number / weight quantized version

#conv_ref = torch.nn.Conv2d(in_channels = 16, out_channels=16, kernel_size = 3, bias = False)
#conv_ref.weight = torch.nn.parameter.Parameter(weight_int)

#output_ref = conv_ref(x)
#print(output_ref)

In [12]:
#### input floating number / weight floating number version

conv_ref = torch.nn.Conv2d(in_channels = 16, out_channels=16, kernel_size = 3, bias = False)
#weight = model.features[3].weight
#mean = weight.data.mean()
#std = weight.data.std()
#conv_ref.weight = torch.nn.parameter.Parameter(weight.add(-mean).div(std))
conv_ref.weight = torch.nn.parameter.Parameter(weight_q)
output_ref = conv_ref(x)
print(output_ref)


tensor([[[[-8.7440e-01,  1.2215e+00,  1.6622e+00,  ..., -9.4380e+00,
           -1.9240e+01, -4.7436e+00],
          [-2.5431e+00, -1.7175e+00, -1.2281e+00,  ..., -8.0976e+00,
           -3.4215e+01, -9.2423e+00],
          [-2.2753e+00, -1.2581e+00, -9.5405e-01,  ..., -1.2030e+01,
           -3.4499e+01, -1.0445e+01],
          ...,
          [-5.0995e+00, -4.8394e+00, -1.0404e+00,  ..., -5.8937e+00,
           -4.8033e+01, -1.1033e+01],
          [-2.4173e+00, -1.1055e+00,  2.1148e-01,  ..., -7.2408e+00,
           -4.1320e+01, -1.2518e+01],
          [-1.0935e+01, -1.4999e+01, -1.2581e+01,  ..., -3.7624e+01,
           -8.7760e+01, -9.3653e+00]],

         [[-1.1996e+01, -7.0394e+00, -8.4347e+00,  ..., -8.1813e+00,
           -7.8023e+00, -1.6607e+01],
          [-6.8502e+00, -1.1905e+00, -1.6521e+00,  ..., -1.1950e+01,
           -1.4484e+01, -1.6004e+01],
          [-7.3602e+00, -5.0270e-01, -6.5749e-03,  ..., -1.2788e+01,
           -1.5259e+01, -1.7736e+01],
          ...,
     

In [13]:
difference = abs( output_ref - output_recovered )
print(difference.mean()) 

tensor(1.1084, device='cuda:0', grad_fn=<MeanBackward0>)
